In [1]:
import tweepy # tweepy module to interact with Twitter
import pandas as pd # Pandas library to create dataframes
from tweepy import OAuthHandler # Used for authentication
from tweepy import Cursor # Used to perform pagination
import json


In [2]:

#using my credentials with academic access.


cons_key = 'XXXX'
cons_secret = 'XXXXX'
acc_token = 'XXXXX'
acc_secret = 'XXXXX'
bearer_token = 'XXXXXX'


# (1). Athentication Function
def get_twitter_auth(): #the authentification to Twitter
    
    
    try:
        consumer_key = cons_key
        consumer_secret = cons_secret
        access_token = acc_token
        access_secret = acc_secret
        
    except KeyError:
        sys.stderr.write("Twitter Environment Variable not Set\n")
        sys.exit(1)
        
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    
    return auth #Function to access the authentication API
def get_twitter_client():   #the client to access the authentification API

 
    auth = get_twitter_auth()
    client = tweepy.API(auth, wait_on_rate_limit=True)
    return client
#Function creating final dataframe
def get_tweets_from_user(ids, page_limit=2, count_tweet=200):

    #ids: the twitter username of a user (company, etc.)
    #page_limit: the total number of pages (max=16)
    #count_tweet: maximum number to be retrieved from a page
        

    client = get_twitter_client()
    
    all_tweets = []
    
    for page in Cursor(client.user_timeline, 
                user_id=ids, 
                count=count_tweet,tweet_mode='extended').pages(page_limit):
        for tweet in page:
            parsed_tweet = {}
            parsed_tweet['source'] = tweet.source #get the source of the tweet (e.g. iphone, android)
            parsed_tweet['all_info'] = tweet._json #get all the rest of the data
                
            all_tweets.append(parsed_tweet)
            if len(all_tweets) >200: #Limit the tweets to 200
                break
    # Create dataframe 
    df = pd.DataFrame(all_tweets)
    # Revome duplicates if there are any
    df2 = pd.json_normalize(df["all_info"]) #Unpack the data in the all_info column
    df2 = df2[["created_at", "id_str", 'full_text', "in_reply_to_user_id_str", "retweet_count",  "favorite_count", "favorited", "retweeted",  "entities.hashtags", 'entities.symbols', 'entities.user_mentions', 'entities.urls', 'user.id', 'user.id_str','retweeted_status.created_at', 'retweeted_status.id', 'retweeted_status.id_str']]
    df3 = pd.concat([df['source'],df2],axis=1) #concat all data in a final dataframe
    return df3

In [84]:
from tqdm import tqdm
ids = pd.read_csv('gilani-2017.tsv',header=None,delim_whitespace=True) #read data 
ids.drop_duplicates(subset=[0],inplace=True,ignore_index=True)
second_dataset = pd.read_csv('cresci-rtbust-2019.tsv',header=None, delim_whitespace=True)
second_dataset.drop_duplicates(subset=[0],inplace=True,ignore_index=True) #drop duplicate ids
ids = pd.concat([ids,second_dataset], axis=0,ignore_index=True) # concat all ids in one file
column_names =  ["source","created_at", "id_str", 'text', "in_reply_to_user_id_str", "retweet_count",  "favorite_count", "favorited", "retweeted",  "entities.hashtags", 'entities.symbols', 'entities.user_mentions', 'entities.urls', 'user.id', 'user.id_str','retweeted_status.created_at', 'retweeted_status.id', 'retweeted_status.id_str']
df_final = pd.DataFrame(columns = column_names) #create a dataframe to concat all the collected tweets
for i in tqdm(range(len(ids[0]))):
    try:
        tweets = get_tweets_from_user(ids[0][i]) #itterate to get all the tweets
    except Exception as e: #to avoid possible errors from missing account and info
        #print(e)
        continue
    df_final = pd.concat([tweets,df_final])
    df_final.reset_index(inplace=True, drop=True)
df_final.to_csv('Final_Data_v3.csv') #save all the data

100%|████████████████████████████████████████████████████████████████████████████| 3392/3392 [1:39:36<00:00,  1.76s/it]


In [3]:
#real users
#repeat same steps as above, just with different ids this time
import pandas as pd
from tqdm import tqdm
column_names =  ["source","created_at", "id_str", 'text', "in_reply_to_user_id_str", "retweet_count",  "favorite_count", "favorited", "retweeted",  "entities.hashtags", 'entities.symbols', 'entities.user_mentions', 'entities.urls', 'user.id', 'user.id_str','retweeted_status.created_at', 'retweeted_status.id', 'retweeted_status.id_str']
df_final = pd.DataFrame(columns = column_names)
user_ids = [234343780, 48008938, 249130209, 1348351605654106118, 86391789, 70340615, 289527193, 1005766052607938560, 62213337, 1495855082734297095, 1976335622, 1066275282653536256]
for i in tqdm(range(len(user_ids))):
    try:
        tweets = get_tweets_from_user(user_ids[i])
    except Exception as e:
        continue
    df_final = pd.concat([tweets,df_final])
    df_final.reset_index(inplace=True, drop=True)
df_final.to_csv('Real_users.csv') #Save to Real_users.csv

100%|██████████| 12/12 [00:18<00:00,  1.55s/it]


In [4]:
#import and auth again 
import tweepy
from tweepy import OAuthHandler


CONSUMER_KEY = cons_key
CONSUMER_SECRET = cons_secret
ACCESS_KEY = acc_token
ACCESS_SECRET = acc_secret
auth = OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
api = tweepy.API(auth)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

#search
api = tweepy.API(auth)
test = api.lookup_users(user_id=user_ids) #take all the required user info using the lookup_users method


In [5]:
json_str = json.loads(json.dumps(test[0]._json)) #unpack the data
df = pd.DataFrame([json_str], columns=json_str.keys()) #create a dataframe
for i in range(len(test)-1): 
    json_str = json.loads(json.dumps(test[i+1]._json))
    df1 = pd.DataFrame([json_str], columns=json_str.keys())
    df = pd.concat([df1,df]) #concat 
df.reset_index(inplace=True,drop=True)
df.to_csv('user_data.csv') #save to csv